In [ ]:
import cv2
import matplotlib.pyplot as plt
import random
import numpy as np
import os

In [ ]:
im = cv2.imread("logos/cogni_solo.png", cv2.IMREAD_UNCHANGED)
plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGRA2RGBA))

In [ ]:
def get_random_cutout(im):
    y = random.randint(0, im.shape[0]//15)
    x = random.randint(0, im.shape[1]//15)
    width = im.shape[1]-x-random.randint(-(im.shape[1]-y)//15, (im.shape[1]-x)//15)
    height = im.shape[0]-y-random.randint(-(im.shape[0]-y)//15, (im.shape[0]-y)//15)
    cutout = im[y:y+height, x:x+width]
    scale = min(random.random()+0.2,1)
    cutout = cv2.resize(cutout, tuple(int(scale*x) for x in im.shape)[:-1][-1::-1])
    if random.random()>0.8:
        blur_rate = max(1,random.randrange(min(cutout.shape[0],cutout.shape[1])//15))
        cutout = cv2.blur(cutout,(blur_rate,blur_rate))
    return cutout

In [ ]:
plt.imshow(cv2.cvtColor(get_random_cutout(im), cv2.COLOR_BGRA2RGBA))

In [ ]:
def get_random_solid_background(im):
    random_color = [random.randrange(256),random.randrange(256),random.randrange(256)]
    return np.full((im.shape[0]+random.randrange(-im.shape[0]//4,im.shape[0]//3),im.shape[1]+random.randrange(-im.shape[1]//4,im.shape[1]//3),3), random_color)

In [ ]:
def overlay_transparent(background, overlay, x, y):

    background_width = background.shape[1]
    background_height = background.shape[0]

    if x >= background_width or y >= background_height:
        return background

    h, w = overlay.shape[0], overlay.shape[1]

    if x + w > background_width:
        w = background_width - x
        overlay = overlay[:, :w]

    if y + h > background_height:
        h = background_height - y
        overlay = overlay[:h]

    if overlay.shape[2] < 4:
        overlay = np.concatenate(
            [
                overlay,
                np.ones((overlay.shape[0], overlay.shape[1], 1), dtype = overlay.dtype) * 255
            ],
            axis = 2,
        )

    overlay_image = overlay[..., :3]
    mask = overlay[..., 3:] / 255.0

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_image
    full_mask = np.full(background.shape, 0.)
    full_mask[y:y+h, x:x+w] = mask
    full_mask*=255
    return background, full_mask

In [ ]:
for i in range(2500):
    cutout = get_random_cutout(im)
    background = get_random_solid_background(cutout)
    x = random.randrange(max(1,background.shape[1]-cutout.shape[1]))
    y = random.randrange(max(1,background.shape[0]-cutout.shape[0]))
    added_image, mask = overlay_transparent(background,cutout,x, y)
    added_image = added_image.astype('uint8')
    
    plt.imshow(cv2.cvtColor(added_image, cv2.COLOR_BGR2RGB))
    plt.show()

    cv2.imwrite(f'output_solo/mask/{i}.png', mask)
    cv2.imwrite(f'output_solo/full/{i}.jpg', added_image)

In [ ]:
def get_random_photo_background(cutout, photo):
    scale = max(cutout.shape[0]/photo.shape[0], cutout.shape[1]/photo.shape[1])
    if scale>1:
        photo = cv2.resize(photo,(photo.shape[0]*scale, photo.shape[1]*scale))
    return photo[0:cutout.shape[0]+random.randrange(-cutout.shape[0]//5,cutout.shape[0]//3), 0:cutout.shape[1]+random.randrange(-cutout.shape[1]//5,cutout.shape[1]//3)]

In [ ]:
random_bgs = os.listdir('random')
for bg_path in random_bgs:
    try:
        cutout = get_random_cutout(im)
        bg = cv2.imread('random/'+bg_path)
        background = get_random_photo_background(cutout, bg)
        added_image, mask = overlay_transparent(background,cutout,random.randrange(max(1,background.shape[1]-cutout.shape[1])),random.randrange(max(1,background.shape[0]-cutout.shape[0])))

        plt.imshow(cv2.cvtColor(added_image, cv2.COLOR_BGR2RGB))
        plt.show()
        cv2.imwrite(f'output_solo/mask/{bg_path}.png', mask)
        cv2.imwrite(f'output_solo/full/{bg_path}.jpg', added_image)
    except Exception as ex:
        print(ex)